# Economic Connectedness
Author: Ioannis Vougias
ioannisvougias@hotmail.com

This notebook contains several visual replications of figures in Python based on the the following papers.

* Chetty, R., Jackson, M.O., Kuchler, T. et al. Social capital I: measurement and associations with economic mobility. Nature 608, 108–121 (2022). https://doi.org/10.1038/s41586-022-04996-4.

* Chetty, R., Jackson, M.O., Kuchler, T. et al. Social capital II: determinants of economic connectedness. Nature 608, 122–134 (2022). https://doi.org/10.1038/s41586-022-04997-3.

------------
Important Terminology for this notebook:

SES: socioeconomic status

Economic Connectedness(EC): The share of high-SES friends among individuals with low-SES. 


#### Below is the list of all the unique files used in this notebook:

For the **first** plot
* 'social_capital_county.csv'
* 'https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json'

For the **second** plot
* 'county_outcomes_simple.csv'

For the **third** plot
* 'zip_covariates.csv', converted file from original 'zip_covariates.dta'
* 'social_capital_zip.csv'

For the **fourth** plot
* 'social_capital_high_school.csv'

For the **fifth** plot
* 'college_characteristics.csv', converted file from original 'college_characteristics.dta'
* 'social_capital_college.csv'

-
In some plots files from previous ones were used.

In [ ]:
#Import the necessary libraries we will use throughout this notebook.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import json
import plotly.express as px
import plotly.graph_objects as go
from urllib.request import urlopen

### 1. The Geography of Social Capital in the United States
This is a replication of Figure 2a of the first paper, making it look like the interactive one we can find available online at https://www.socialcapital.org/.

The map is constructed using Plotly. The data are displayed per county. When the point hovers each county, it  displays the name of the county, the state it belongs to, the FIPS code of the county and the economic connectedness of the county. If there are no data for a particular county, it is painted with a distinct color (gold) and the economic connectedness is given as "NA".

Data for social capital can be found at the [Social Capital Atlas Datasets](https://data.humdata.org/dataset/social-capital-atlas).


The share of high-SES friends among individuals with low SES — which we term economic connectedness — is among the strongest predictors of upward income mobility identified to date.

We download the csv file named 'Social Capital Atlas - US Counties.csv', from the [Social Capital Atlas Datasets](https://data.humdata.org/dataset/social-capital-atlas) website. 

After reading the relevant READme file, we will use 'Social Capital Atlas - US Counties.csv', because it contains the Economic Connectedness per county (column: ec_county)

In [ ]:
#Read the file and create dataframe named sc_county.
sc_county = pd.read_csv('social_capital_county.csv')

#Show the first 5 rows
sc_county.head(5)

We read the documentation of the csv file and we find that the column 'county' represents a 5-digit code for every county in USA. Unfortunately in this dataset, if a county-code starts with '0', it is ommited. So we will make every county code 5-digit.

In [ ]:
#Edit the column 'county' as string.
sc_county["county"] = sc_county["county"].astype('string')

#Make the ec_county column float and round it to two decimals.
sc_county['ec_county'] = sc_county['ec_county'].astype('float')
sc_county['ec_county'] = sc_county['ec_county'].round(2)

#Use the zfill method to make every county-code 5-digit.
sc_county["county"] = sc_county["county"].str.zfill(5)


In [ ]:
#Create quantiles based on Economic Connectedness.
sc_county['quantile'] = pd.qcut(sc_county['ec_county'], q=10, precision=3)

#Create a list containing the labels we want to use in our graph later.
label_list = ['<0.58','0.58-0.67','0.67-0.72','0.72-0.76', 
              '0.76-0.81', '0.81-0.85','0.85-0.90',
              '0.90-0.97','0.97-1.06','>1.06']

#Create a new column matching the quantile a county is in, with the proper label.
sc_county['quantile_labels'] = pd.qcut(sc_county['ec_county'], q=10, labels = label_list)

#Make columns 'quantile_labels','quantile' and 'ec_county' strings for better manipulation.
sc_county['quantile_labels'] = sc_county['quantile_labels'].astype('string')
sc_county['quantile'] = sc_county['quantile'].astype('string')

In [ ]:
#Fill the NaN values with the string value 'NA'.
sc_county['quantile_labels'] = sc_county['quantile_labels'].fillna('NA')

#Create a new column with ec_county value as string for better manipulation in visualization
sc_county['ec_county_str'] = sc_county['ec_county']
sc_county['ec_county_str'] = sc_county['ec_county_str'].astype('string')
sc_county['ec_county_str'] = sc_county['ec_county_str'].fillna('NA')

In [ ]:
#Load from this github repository the counties and their FIPS codes.
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
    
#Create a custom color list starting from red as the lowest quantile of EC and blue the highest.
color_list = {'<0.58': '#67001f',
              '0.58-0.67': '#b6202f',
              '0.67-0.72': '#dd6f59',
              '0.72-0.76':  '#f7b799',
              '0.76-0.81': '#fae7dc',
              '0.81-0.85': '#e2edf3',
              '0.85-0.90':  '#a7d0e4',
              '0.90-0.97': '#559ec9',
              '0.97-1.06': '#256baf',
              '>1.06' : '#053061',
              'NA':'#FFD700'}

#Create a list for category order.
category_list = ['>1.06', '0.97-1.06','0.90-0.97','0.85-0.90', 
                 '0.81-0.85', '0.76-0.81','0.72-0.76', 
                 '0.67-0.72', '0.58-0.67','<0.58']
  
#Order the column based on the category_list. 
category_order_dict = {'quantile_labels': category_list}  

#Data we want to display when map value is hovered.
hover_data_dict = {
        'county': True,
        'ec_county_str': True,
        'quantile_labels': False
        }
    
#Create the plot.    
fig = px.choropleth(sc_county, geojson=counties, 
                           locations='county', 
                           color='quantile_labels',
                           color_discrete_map = color_list,
                           category_orders = category_order_dict,
                           scope="usa",
                           hover_name = sc_county['county_name'],
                           hover_data = hover_data_dict,
                           labels = {'ec_county_str' : 'Economic Connectedness',
                                     'quantile_labels':'Economic connectedness'}
                   )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})


fig.show()

### 2. Economic Connectivity and Outcomes

This is a replication on the [Figure 4](https://www.nature.com/articles/s41586-022-04996-4/figures/4) of the first paper. The figure is a scatter plot of upward income mobility against economic connectedness (EC) for the 200 most populous US counties. The income mobility is obtained from the [Opportunity Atlas](https://www.nber.org/papers/w25147), whose replication data can be found [here](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/NKCQM1).

In [ ]:
#Find the 200 most populous US counties.
sc_county['pop2018'] = sc_county['pop2018'].astype(float)
sc_county_n200 = sc_county.nlargest(200, ['pop2018'])

#Make a string column named 'full_county' to merge later the dataframes.
sc_county_n200["full_county"] = sc_county_n200["county"].astype('str') 

#Print the 5 most populous US counties.
sc_county_n200.head(5)

We download the csv file named 'county_outcomes.csv', from the [Replication Data for: The Opportunity Atlas: Mapping the Childhood Roots of Social Mobility](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/NKCQM1) website. It contains the predicted household income rank in adulthood for children in the 1978–1983 birth cohorts with parents at the 25th percentile of the national income distribution.

In [ ]:
#Read the csv file and create the county_outcomes df.
county_outcomes = pd.read_csv('county_outcomes_simple.csv')

#In the sc_county df we have a 5-digit code for every county.
#In the county_outcomes df we have a 2-digit code for the state and a 3-digit code for the county.

#Make the county column string type.
county_outcomes["county"] = county_outcomes["county"].apply(str)

#Make all the county codes 3-digit, because 0 in the beggining is ommited.
county_outcomes["county"] = county_outcomes["county"].str.zfill(3)

#Same for the states.
county_outcomes["state"] = county_outcomes["state"].apply(str)
county_outcomes["state"] = county_outcomes["state"].str.zfill(2)

#Merge the 2-digit code of the state with the 3-digit code of the county to have the same 5-digit code of the
#sc_county df. Create a new column named 'full_county' with the 5-digit codes.
county_outcomes['full_county'] = county_outcomes["state"] + county_outcomes["county"] 

We will create a new copy of the 'county_outcomes' df, named 'upward_mobility', to only get the columns we need.

We need the county code 'full_county' we created, to merge the df of the EC with the one with the upward mobility ranking.

Based on the documentation (CodeBook-for-Table-2.pdf) we downloaded from the same website,
we want the column ('kfr_pooled_pooled_p25') with the upward mobility ranking of all the races (pooled) and all the genders (pooled).

In [ ]:
#Create the df with the county code and the upward mobility ranking.
upward_mobility = county_outcomes[['full_county','kfr_pooled_pooled_p25']].copy()

#Multiply by 100 to get the percentage.
upward_mobility['Rank_percentage'] = upward_mobility['kfr_pooled_pooled_p25'] * 100

upward_mobility["full_county"] = upward_mobility["full_county"].apply(str)

#Print the first 5 rows.
upward_mobility.head(5)

In [ ]:
#We merge the df with the EC and the df with the upward mobility.
scatterfinal = pd.merge(sc_county_n200, upward_mobility,on='full_county',how='left')

scatterfinal["county_name"] = scatterfinal["county_name"].astype('string')

scatterfinal = scatterfinal[['county', 'county_name', 'ec_county', 'kfr_pooled_pooled_p25', 'Rank_percentage']].copy()

In [ ]:
#Find the EC and the upward mobility of 5 counties we want to visualize in our scatterplot
scatterfinal.loc[scatterfinal['county_name'] == 'San Francisco, California']
scatterfinal.loc[scatterfinal['county_name'] == 'New York, New York']
scatterfinal.loc[scatterfinal['county_name'] == 'Salt Lake, Utah']

#Indianapolis official county name is Marion
scatterfinal.loc[scatterfinal['county_name'] == 'Marion, Indiana'] 

#Minneapolis official county name is Hennepin
scatterfinal.loc[scatterfinal['county_name'] == 'Hennepin, Minnesota'] 

After a quick google search we found out that Indianapolis is on Marion, Indiana county and Minneapolis is on Hennepin, Minnesota county.

In [ ]:
#Set style to darkgrid
sns.set_style('darkgrid')

#Make the scatterplot using lmplot
snsfig = sns.lmplot(x='ec_county', y='kfr_pooled_pooled_p25', data=scatterfinal, aspect=2)

#Set labels on axis
snsfig.set(
    xlabel = 'Economic Connectedness', 
    ylabel="""    Predicted Household Income Rank
              For Children With Parents at 25th Percintile""")

#Annotate San Fransisco
snsfig.ax.annotate('San Fransisco',
                xy = (1.31,0.5),
                xytext=(1.31,0.375),
                xycoords='data',  
                arrowprops=dict(facecolor='black',width=2,headwidth=10.0,headlength=10.0,shrink=0))

#Annotate New York City
snsfig.ax.annotate('New York City',
                xy = (0.827,0.42),
                xytext=(0.827,0.5),
                xycoords='data',  
                arrowprops=dict(facecolor='black',width=2,headwidth=10.0,headlength=10.0,shrink=0))

#Annotate Salt Lake City
snsfig.ax.annotate('Salt Lake City',
                xy = (0.965,0.455),
                xytext=(0.965,0.51),
                xycoords='data',  
                arrowprops=dict(facecolor='black',width=2,headwidth=10.0,headlength=10.0,shrink=0))

#Annotate Indianapolis
snsfig.ax.annotate('Indianapolis',
                xy = (0.642,0.345),
                xytext=(0.7,0.33),
                xycoords='data',  
                arrowprops=dict(facecolor='black',width=2,headwidth=10.0,headlength=10.0,shrink=0))

#Annotate Minneapolis
snsfig.ax.annotate('Minneapolis',
                xy = (0.975,0.427),
                xytext=(1.05,0.375),
                xycoords='data',  
                arrowprops=dict(facecolor='black',width=2,headwidth=10.0,headlength=10.0,shrink=0))


#Set xlim
snsfig.set(xlim=(0.4,1.4))

### 3. Upward Income Mobility, Economic Connectedness, and Median House Income

This is a replication of the [Figure 6 of the first paper](https://www.nature.com/articles/s41586-022-04996-4/figures/6). The figure is a scatter plot of economic connectedness (EC) against median household income. We combine data from replication package of the papers (ACS and Opportunity Atlas) with data from the Social Capital Atlas Datasets. The color of the dots corresponds to the child's income rank in adulthood given that the parents' income is in the 25th percentile. The colors correspond to five intervals, which are the quintiles dividing our data. We need the compiled csv file 'zip_covariates.csv' to get the median household income and the upward mobility for each zip code. We got the file from the public data of the replication package, converting it to csv for better manipulation in python.

In [ ]:
#Convert zip_covariates.dta to csv file
data = pd.io.stata.read_stata('zip_covariates.dta')
data.to_csv('zip_covariates.csv')


#Use the zip_covariates.csv file which contains upward mobility and median household income per zip code.
upward_mobility = pd.read_csv('zip_covariates.csv', index_col=0)

#Use the bin edges we want for our scatterplot.
bins=[0, 0.38 ,0.41, 0.44, 0.48, 1]

#Create new column named 'quantile' to divide our data into 5 bins.
upward_mobility['quantile'] = pd.cut(upward_mobility['kfr_pooled_pooled_p25'], bins = bins)

#Print 5 first rows.
upward_mobility.head(5)

In [ ]:
#Keep only the columns we need.
upward_mobility = upward_mobility[['zip', 'kfr_pooled_pooled_p25', 'med_inc_2018', 'pop2018', 'quantile']]

#Print the df.
upward_mobility

In [ ]:
#Read the csv containing the Economic Connectedness per zip code from Social Capital Atlas Datasets.
social_capital_zip = pd.read_csv('social_capital_zip.csv')

#Keep only the rows we need.
social_capital_zip =  social_capital_zip[['zip', 'ec_zip']]

In [ ]:
#Merge the two dataframes .
ec_vs_um = pd.merge(social_capital_zip, upward_mobility, on = 'zip', how = 'left')

#Print the merged dataframe.
ec_vs_um.head(5)

In [ ]:
#Remove rows containing NaN values from the columns we need information to be non NaN.
ec_vs_um = ec_vs_um[ec_vs_um['ec_zip'].notna()]
ec_vs_um = ec_vs_um[ec_vs_um['med_inc_2018'].notna()]
ec_vs_um = ec_vs_um[ec_vs_um['kfr_pooled_pooled_p25'].notna()]
ec_vs_um = ec_vs_um[ec_vs_um['quantile'].notna()]

ec_vs_um.head(5)

In [ ]:
#Keep only the zip codes where median household income is greater than 30.000$ and lower than 100.000$.
ec_vs_um  = ec_vs_um[ec_vs_um['med_inc_2018'] >= 30000]
ec_vs_um  = ec_vs_um[ec_vs_um['med_inc_2018'] <= 100000]

#Drop zip codes with population that is included in the 10th percentile of the whole dataset. The least populated.
ec_vs_um['pop_deciles'] = pd.qcut(ec_vs_um['pop2018'], 10, labels = False)
ec_vs_um  =  ec_vs_um[ec_vs_um['pop_deciles'] != 0]

#Convert now quantile column to string for label manipulation.
ec_vs_um['quantile'] = ec_vs_um['quantile'].astype('string')

In [ ]:
#Create our labels for the scatterplot.
q_labels = ['> 48','44 - 48','41 - 44','38 - 41','< 38']

#Create a function that converts the quantile span to a specific label.
def compute_label(df):    
    if (df['quantile'] == '(0.0, 0.38]'):
        return q_labels[4]
    elif (df['quantile'] == '(0.38, 0.41]'):
        return q_labels[3]
    elif (df['quantile'] == '(0.41, 0.44]'):
        return q_labels[2]
    elif (df['quantile'] == '(0.44, 0.48]'):
        return q_labels[1]
    elif (df['quantile'] == '(0.48, 1.0]'):
        return q_labels[0]

#Apply the fucntion and create a new column named 'Upward Mobility'.    
ec_vs_um['Upward Mobility'] = ec_vs_um.apply(compute_label, axis = 1)   

#Print the df.
ec_vs_um.head(5)

In [ ]:
#Create our scatterplot using seaborn. 

# Make the scatterplot with better analysis
plt.rcParams['figure.dpi'] = 150
plt.rcParams['savefig.dpi'] = 150

#X-axis has median household income per zip codes.
#y-axis Economic Connectedness per zip codes.

#Hue is the quantile that each zip code is included based on upward moblity.

#Create custom color pallete from red to blue.
colors = ["#191970","#009acd","#F2D2BD","#FFA500","#cd0000"]

sns.set_style('darkgrid')
ax = sns.scatterplot(data = ec_vs_um, x='med_inc_2018', y='ec_zip', 
                     hue = 'Upward Mobility',palette = colors, hue_order = q_labels, alpha=0.75)

ax.set_xticks([30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000])

#Set label names.
ax.set(xlabel = 'Median Household Income in ZIP codes (US$)',
       ylabel = 'Economic Connectedness')

#Legend attributes.
plt.legend(loc='lower right', fontsize=5)

plt.rcParams["figure.autolayout"] = True

### 4. Friending Bias and Exposure by High School

This is a replication of [Figure 5a of the second paper](https://www.nature.com/articles/s41586-022-04997-3/figures/5). The figure depicts the Socio-Economic Status (SES) of parents against the friending bias of students of low SES, with data from the Social Capital Atlas Datasets. 

Both $x$ and $y$ axis are percentages and the $y$ axis are reversed.

After reading the documentation, the columns we need from the csv file 'social_capital_high_school' to create the plot are:

* 'high_school': the code for each high school

* 'high_school_name': the name for each high school

* 'exposure_parent_ses_hs': Mean exposure to high-parental-SES individuals by high school for
low-parental-SES individuals:

* 'bias_parent_ses_hs': Economic connectedness with parental SES divided by Mean exposure to high-parental-SES individuals by high school for low-parental-SES individuals, all subtracted from one

In [ ]:
#Read the csv file named 'social_capital_high_school.csv' which contains the information we need and create new df.
sc_high_school = pd.read_csv('social_capital_high_school.csv')

#Create a new column to calculate Share of high-parental-SES students %.
sc_high_school['parent_exposure'] =  sc_high_school['exposure_parent_ses_hs']

#Create a new column to calculate Friending Bias among low parental-SES students %.
sc_high_school['parent_bias'] =  sc_high_school['bias_parent_ses_hs']

#Keep only the not NaN values of the columns we created.
sc_high_school = sc_high_school[sc_high_school['parent_exposure'].notna()]
sc_high_school = sc_high_school[sc_high_school['parent_bias'].notna()]

#Make the proper calculations as per description.
sc_high_school['parent_exposure'] = (sc_high_school['parent_exposure'] / 2) * 100  
sc_high_school['parent_bias'] =  sc_high_school['parent_bias'] * 100

In [ ]:
#Create the data with the high schools we want to show on the scatterplot.
high_school_codes = ["00941729","060474000432","170993000942","170993001185",
                     "170993003989","171449001804","250327000436","360009101928",
                     "370297001285","483702004138","250843001336","062271003230",
                     "010237000962","00846981","00852124"]

# Create a dataframe with the data above.
high_schools_to_show = pd.DataFrame(high_school_codes, columns=['high_school'])
  
#Merge the new df with the social_high_school to get all the information for the high schools we want.
high_schools_to_show =  pd.merge(high_schools_to_show, sc_high_school,on=['high_school'],how='left')

#Keep only the columns we need.
high_schools_to_show = high_schools_to_show[['high_school','high_school_name','parent_exposure','parent_bias']].copy()

#Print the df.
high_schools_to_show

We will keep only the values based on the limits we will set later, to avoid ggplot removing automatically them and showing us a warning.

In [ ]:
#for x-axis, 0 to 90 are the x-axis limits we will set.
sc_high_school = sc_high_school[sc_high_school['parent_exposure'] >= 0]
sc_high_school = sc_high_school[sc_high_school['parent_exposure'] <= 90]

#for y-axis, -15 to 30 are the y-axis limits we will set.
sc_high_school = sc_high_school[sc_high_school['parent_bias'] >= -15]
sc_high_school = sc_high_school[sc_high_school['parent_bias'] <= 30]

In [ ]:
#Add Jitter to the high school name labels so they don't overlap on the plot.

#the numbers are approximates for a more clear scatterplot.

high_schools_jitter = high_schools_to_show.copy()

#Dalton School
high_schools_jitter.at[0,'parent_exposure'] = 67

#Berkeley HS
high_schools_jitter.at[1,'parent_bias'] = 15

#Lane Technical HS
high_schools_jitter.at[2,'parent_exposure'] = 53
high_schools_jitter.at[2,'parent_bias'] = -4

#Lincoln Park HS
high_schools_jitter.at[3,'parent_exposure'] = 44

#Payton College Preparatory HS
high_schools_jitter.at[4,'parent_bias'] = 4
high_schools_jitter.at[4,'parent_exposure'] = 64

#Evanston Twp HS
high_schools_jitter.at[5,'parent_exposure'] = 70

#Cambridge Rindge And Latin
high_schools_jitter.at[6,'parent_bias'] = 9

#Brooklyn Technical HS
high_schools_jitter.at[7,'parent_bias'] = -2

#West Charlotte HS
high_schools_jitter.at[8,'parent_exposure'] = 14

#New Bedford HS
high_schools_jitter.at[10,'parent_bias'] = 3

#John L Leflore Magnet School
high_schools_jitter.at[12,'parent_bias'] = -1.5

#Bishop Gorman HS
high_schools_jitter.at[13,'parent_bias'] = 5
high_schools_jitter.at[13,'parent_exposure'] = 86

#Phillips Exeter Academy
high_schools_jitter.at[14,'parent_exposure'] = 84

In [ ]:
#Import plotnine to create a ggplot style scatterplot.
import plotnine as p9

#Make the plot.
(p9.ggplot(data=sc_high_school, mapping=p9.aes(x='parent_exposure', y='parent_bias')) 
 
 + p9.geom_point(alpha = 0.12)
 
 + p9.scales.scale_y_continuous(limits = (30, -15), breaks = (30, 20, 10, 0, -10), trans = "reverse")
 
 + p9.scales.scale_x_continuous(limits = (0, 90), breaks = (0, 20, 40, 60, 80, 100)) 
 
 + p9.labels.labs(x = "Share of high-parental-SES students (%)", 
                  y = "Friending bias among low-parental-SES students (%)") 
 
 + p9.themes.theme(axis_text_x = p9.element_text(size = 8),
                   axis_text_y = p9.element_text(size = 8),
                   axis_title = p9.element_text(size = 8))
 
 + p9.geom_point(data = high_schools_to_show,color = "aquamarine", size = 3, alpha = 0.8)
 
 + p9.geom_label(data = high_schools_jitter, color = "black",
                        alpha = 0.9, label_size = 0.1, label_padding = 0.5, 
                        mapping = p9.aes(label = 'high_school_name'), size = 5.7, 
                        position= p9.position_jitter(width= 3,height=5), nudge_x= -2,nudge_y=2)
)  

### 5. Friending Bias vs. Racial Diversity

This is a replication of [Extended Data Figure 3](https://www.nature.com/articles/s41586-022-04997-3/figures/9) of the second paper. The figure depicts friending bias against racial diversity. Racial diversity is defined by the [Herfindahl-Hirschman Index (HHI)](https://en.wikipedia.org/wiki/Herfindahl%E2%80%93Hirschman_index), borrowed from investing. Translated here, it is $ 1−\sum_{i}{s_i}^2$, where $s_i$ is the fraction of race/ethnicity $i$ (Black, White, Asian, Hispanic, Native American).

As you can see, the figure contains two scatter plots with their respective regression lines, one for college data and the other for neighborhood data. Each of the two plots displays binned data (that's why you don't see loads of dots and diamonds). The bins are produced by dividing the $x$-axis into ventiles (i.e., 5 percentile point bins); then we plot the mean of the $y$-axis variable against the appropriate mean of the $x$-axis variable in each ventile. 

The mean of the $x$-axis variable, the HHI index, is the weighted mean of HHI:

* For the college plot, the weights are given by the mean number of students per cohort.

* For the neighborhood plot, the weights are given by the number of children with below-national-median parental household income.

The $y$-axis variable:

* For the college plot, it is the mean of the college friending bias.

* For the neighborhood plot, it is the mean of the neighborhood friending bias.

#### 5.1 Neighborhood plot

We will make the proper data manipulation to create the data for the Neighborhood plot (Orange line).

As we need the fractions of race/ethnicity, we will use again the csv file we converted from the social capital replication package.

Spefically we need to use the columns below:
* 'share_white_2018' for the fraction of the race 'white'
* 'share_black_2018' for the fraction of the race 'black'
* 'share_natam_2018' for the fraction of the ethnicity 'native American'
* 'share_asian_2018' for the fraction of the ethnicity 'asian'
* 'share_hispanic_2018' for the fraction of the ethnicity 'hispanic'

To calculate the HHI per zip code.

We also need the column 'num_below_p50', so we can calculate the appropriate weighted mean.

In [ ]:
#Read the file
racial_diversity = pd.read_csv('zip_covariates.csv', index_col=0)

#Print first 5 rows
racial_diversity.head(5)

We need again the csv file named 'social_capital_zip.csv' because we ant to use the column:

* 'nbhd_bias_zip', containing the neighborhood friending bias per zip code

In [ ]:
#Read the file
sc_zip = pd.read_csv('social_capital_zip.csv')

#Print first 5 rows
sc_zip.head(5)

In [ ]:
#Merge the two Dataframes, named friending bias vs racial diversity
fb_vs_rd = pd.merge(sc_zip, racial_diversity, on = 'zip', how = 'left')

#Print first 5 rows
fb_vs_rd.head(5)

In [ ]:
#Calculate the hhi based on the mathematical formula on the description
hhi = 1 - ((fb_vs_rd['share_white_2018'] ** 2) +  
           (fb_vs_rd['share_black_2018'] ** 2) + 
           (fb_vs_rd['share_natam_2018'] ** 2) +
           (fb_vs_rd['share_asian_2018'] ** 2) +
           (fb_vs_rd['share_hawaii_2018'] ** 2) +
           (fb_vs_rd['share_hispanic_2018'] ** 2))

In [ ]:
#Create a new column with the hhi per zip code
fb_vs_rd['Herfindahl-Hirschman-Index'] = hhi

#Keep the columns we need in a copy of the dataset
fb_vs_rd = fb_vs_rd[['zip','nbhd_bias_zip','Herfindahl-Hirschman-Index','num_below_p50_x']].copy()

In [ ]:
#Rescale neighborhood friending bias to 100
fb_vs_rd['nbhd_bias_zip'] = fb_vs_rd['nbhd_bias_zip'] * 100

In [ ]:
#Create a column with 20 bins, ventiles, based on the HHI.
fb_vs_rd['bin'] = pd.qcut(fb_vs_rd['Herfindahl-Hirschman-Index'], 20, labels = False)

#Print first 5 rows
fb_vs_rd.head(5)

Now we reached the pre-final phase. 

Basically we need to create a dataframe with 20 points for the neighborhood plot.

Both the plots are the mean of the  𝑦 -axis variable against the appropriate mean of the  𝑥 -axis variable in each ventile as per description.

* In this case the y-axis variable, is the mean of the neighborhood friending bias.


* For the x-axis, it is the weighted mean of the HHI. The weights are given by the number of children with below-national-median parental household income ('num_below_p50' column).

So we need to calculate both the above.

In [ ]:
#Calculate the weighted mean of the HHI.

#Group the data by bin, aka ventile.
grouped_x = fb_vs_rd.groupby('bin')

#Create a function to calculate the weighted mean.
def wavg(group):
    d = group['Herfindahl-Hirschman-Index']
    w = group['num_below_p50_x']
    return (d * w).sum() / w.sum()

#Apply the function to the grouped data.
nei_x = grouped_x.apply(wavg)

#Calculate the mean of the neighborhood friending bias.

#Group the data by bin, aka ventile.
grouped_y = fb_vs_rd.groupby('bin')

#Create a function to calculate the mean.
def binmean(group):
    d = group['nbhd_bias_zip']
    return d.sum() / d.count()

#Apply the function to the grouped data.
nei_y = grouped_y.apply(binmean)

#Create a dataframe with the x-axis points.
nei_df_x = pd.DataFrame (nei_x, columns = ['Point'])
nei_df_x = nei_df_x.reset_index(level=0)

#Create a dataframe with the y-axis points.
nei_df_y = pd.DataFrame (nei_y, columns = ['Point'])
nei_df_y = nei_df_y.reset_index(level=0)

#Merge the two dataframes.
nei_df = pd.merge(nei_df_x, nei_df_y, on = 'bin', how = 'inner')

#Create a column named type, so we can use it later to plot based on hue.
nei_df['Type'] = 'Neighborhood'

#Print the dataframe
nei_df

'nei_df', contains all the points we need to plot the neighborhood plot. We will merge it later with the counterpart of the college plot to create our final dataframe.

#### 5.2 College plot

We will make the proper data manipulation to create the data for the College plot (Blue line).

As we need the fractions of race/ethnicity, we will use the converted .dta file, to csv, named 'college_characteristics.csv' from the social capital replication package.

Spefically we need to use the columns below:
* 'black_share_fall_2000' for the fraction of the race 'black'
* 'asian_or_pacific_share_fall_2000' for the fraction of the race 'asian'
* 'hisp_share_fall_2000' for the fraction of the ethnicity 'hispanic'

To calculate the HHI per college.

We notice that there is no column for the white race. So we have to calculate it.

We also need the column ''mean_students_per_cohort', so we can calculate the appropriate weighted mean.

In [ ]:
#Convert college_characteristics.dta to csv file
data = pd.io.stata.read_stata('college_characteristics.dta')
data.to_csv('college_characteristics.csv')


#Read the file
college_characteristics = pd.read_csv('college_characteristics.csv', index_col=0)

#Print first 5 rows
college_characteristics.head(5)

We will also use the file 'social_capital_college.csv' containing the friending bias per college. 

In [ ]:
#Read the file
sc_college = pd.read_csv('social_capital_college.csv')

#Print first 5 rows
sc_college.head(5)

In [ ]:
#Merge the two dataframes
college_merged = pd.merge(sc_college, college_characteristics, on = 'college', how = 'inner')

#Print first 5 rows
college_merged.head(5)

In [ ]:
#Create a new column with the fraction for the white race.
college_merged['white_share_fall_2000'] =  1 - college_merged['hisp_share_fall_2000'] - college_merged['black_share_fall_2000'] - college_merged['asian_or_pacific_share_fall_2000']

In [ ]:
#Calculate the hhi
hhi = 1 - ((college_merged['white_share_fall_2000'] ** 2) +  
           (college_merged['black_share_fall_2000'] ** 2) + 
           (college_merged['asian_or_pacific_share_fall_2000'] ** 2) +
           (college_merged['hisp_share_fall_2000'] ** 2))

#Create a new column with the hhi per college
college_merged['Herfindahl-Hirschman-Index'] = hhi

#Keep the columns we need in a copy of the df
college_merged = college_merged[['zip_x','bias_own_ses_college','Herfindahl-Hirschman-Index','mean_students_per_cohort_y']].copy()

In [ ]:
#Rescale the college friending bias to 100
college_merged['bias_own_ses_college'] = college_merged['bias_own_ses_college'] * 100

In [ ]:
#Create a column with 20 bins, ventiles, based on the HHI
college_merged['bin'] = pd.qcut(college_merged['Herfindahl-Hirschman-Index'], 20, labels = False)

#Print first 5 rows
college_merged.head(5)

Now we reached the next pre-final phase. 

Basically we need to create a dataframe with 20 points for the college plot.

Both the plots are the mean of the  𝑦 -axis variable against the appropriate mean of the  𝑥 -axis variable in each ventile as per description.

* In this case the y-axis variable, it is the mean of the college friending bias.


* For the x-axis, it is the weighted mean of the HHI. the weights are given by the mean number of students per cohort.

So we need to calculate both the above.

In [ ]:
#Calculate the weighted mean of the HHI.

#Group data per bin, aka ventile
grouped_x2 = college_merged.groupby('bin')

#Create function that calculates the weighted mean.
def wavg(group):
    d = group['Herfindahl-Hirschman-Index']
    w = group['mean_students_per_cohort_y']
    return (d * w).sum() / w.sum()

#Apply the function to the binned data.
coll_x = grouped_x2.apply(wavg)


#Calculate the mean of the college friending bias.

#Group data per bin, aka ventile
grouped_y2 = college_merged.groupby('bin')

#Create function that calculates the mean.
def binmean(group):
    d = group['bias_own_ses_college']
    return d.sum() / d.count()

#Apply the function to the binned data.
coll_y = grouped_y2.apply(binmean)


#Create a dataframe with the x-axis points.
coll_df_x = pd.DataFrame (coll_x, columns = ['Point'])
coll_df_x = coll_df_x.reset_index(level=0)

#Create a dataframe with the y-axis points.
coll_df_y = pd.DataFrame (coll_y, columns = ['Point'])
coll_df_y = coll_df_y.reset_index(level=0)

#Merge the two dataframes.
coll_df = pd.merge(coll_df_x, coll_df_y, on = 'bin', how = 'inner')

#Create a column named type, so we can use it later to plot based on hue.
coll_df['Type'] = 'College'

#Print the dataframe
coll_df

'coll_df', contains all the points we need to plot the college plot. Now we will merge it  with the 'neig_df' counterpart to create our final dataframe.

In [ ]:
#Append the nei_df to the coll_df
binscatter_df = coll_df.append(nei_df, ignore_index=True)

#Print the final df we will use to plot the scatterplot
binscatter_df

In [ ]:
#Plot the scatterplot based on hue(Neighborhood or College)

#Set style to darkgrid
sns.set_style('darkgrid')

#Make the scatterplot using lmplot
binscatter = sns.lmplot(x='Point_x', y='Point_y', hue = 'Type', data = binscatter_df,
                        aspect=1.5, markers=["D","o"])

binscatter.set(xticks = (0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7,0.8))
binscatter.set(yticks = (0, 5, 10, 15, 20, 25))

#Set labels on axis
binscatter.set(xlabel = 'Racial Diversity (Herfindahl-Hirschman Index) in Group', 
               ylabel = 'Friending Bias among Low-SES Individuals (%)')

#Position the legend on the upper left
plt.legend(loc='upper left')

#Remove hue legend
binscatter._legend.remove()